## Import Libs

## TO DO: Create param cell to define param file, input dirs, output dirs and output file

In [ ]:
# DataFile
# OutputDirectory


In [1]:
import os
from pikepdf import Pdf
import requests
import pandas as pd
from pandas.io.json import json_normalize
import json
from PyPDF2 import PdfFileWriter, PdfFileReader


## Split Files

Create a dict to split every page of the PDF File

TO DO: 
- Optimize file dictionary
- Move to pyPDF???

In [11]:
# a dictionary mapping PDF file to original PDF's page range
file2pages = {
    0: [0, 1], # 1st splitted PDF file will contain the pages from 0 to 9 (9 is not included)
    1: [1, 2], # 2nd splitted PDF file will contain the pages from 9 (9 is included) to 11
    2: [2, 3], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    3: [3, 4], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    4: [4, 5], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    5: [5, 6], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    6: [6, 7], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    7: [7, 8], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    8: [8, 9], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    9: [9, 10], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    10: [9, 10], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)

    11: [10,11], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    12: [11, 12], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    13: [12, 13], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    14: [13, 14], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    15: [14, 15], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    16: [15, 16], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    17: [16, 17], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    18: [17, 18], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    19: [18, 19], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)
    20: [19, 20], # 3rd splitted PDF file will contain the pages from 11 until the end or until the 100th page (if exists)


}

## Setup File

Setup file and load it.
Create individual files
Iterate over the original PDF file and write it to disk

TO DO: Use parameters

In [3]:
# the target PDF document to split
##filename = "C:\Users\hcquinta\OneDrive - Microsoft\Documents 1\My MSFT\[personal]\chasing\source\Catalogo 26.pdf"
filename = "C:\\Users\hcquinta\\OneDrive - Microsoft\Documents 1\\My MSFT\\[personal]\\chasing\\source\\Catalogo 26.pdf"

In [13]:
# load the PDF file
pdf = Pdf.open(filename)

In [14]:
# make the new splitted PDF files
new_pdf_files = [ Pdf.new() for i in file2pages ]
# the current pdf file index
new_pdf_index = 0

In [ ]:
# iterate over all PDF pages
for n, page in enumerate(pdf.pages):
    if n in list(range(*file2pages[new_pdf_index])):
        # add the `n` page to the `new_pdf_index` file
        new_pdf_files[new_pdf_index].pages.append(page)
        print(f"[*] Assigning Page {n} to the file {new_pdf_index}")
    else:
        # make a unique filename based on original file name plus the index
        name, ext = os.path.splitext(filename)
        output_filename = f"{name}-{new_pdf_index}.pdf"
        # save the PDF file
        new_pdf_files[new_pdf_index].save(output_filename)
        print(f"[+] File: {output_filename} saved.")
        # go to the next file
        new_pdf_index += 1
        # add the `n` page to the `new_pdf_index` file
        new_pdf_files[new_pdf_index].pages.append(page)
        print(f"[*] Assigning Page {n} to the file {new_pdf_index}")

# save the last PDF file
name, ext = os.path.splitext(filename)
output_filename = f"{name}-{new_pdf_index}.pdf"
new_pdf_files[new_pdf_index].save(output_filename)
print(f"[+] File: {output_filename} saved.")

In [35]:
# from PyPDF2 import PdfFileWriter, PdfFileReader
fileList = list()
inputpdf = PdfFileReader(open(filename, "rb"))


for i in range(inputpdf.numPages):
    output = PdfFileWriter()
    output.addPage(inputpdf.getPage(i))
    with open("document-page%s.pdf" % i, "wb") as outputStream:
        output.write(outputStream)
    fileList.append("document-page%s.pdf" % i)

print(fileList)

['document-page0.pdf', 'document-page1.pdf', 'document-page2.pdf', 'document-page3.pdf', 'document-page4.pdf', 'document-page5.pdf', 'document-page6.pdf', 'document-page7.pdf', 'document-page8.pdf', 'document-page9.pdf']


In [37]:
 
for e in range(len(fileList)):
    print("Iterate through list:",fileList[e])

Iterate through list: document-page0.pdf
Iterate through list: document-page1.pdf
Iterate through list: document-page2.pdf
Iterate through list: document-page3.pdf
Iterate through list: document-page4.pdf
Iterate through list: document-page5.pdf
Iterate through list: document-page6.pdf
Iterate through list: document-page7.pdf
Iterate through list: document-page8.pdf
Iterate through list: document-page9.pdf


# Request Analysis

Analysis is done async, we need to API calls, 
1. To request the analysis
2. To request the results of the analysis

- Setup the cognitive service and send PDF file for analysis
- Get the request Id sent back by the service located in the headers (APIM Request Id)


TO DO: 
- Loop across all the PDF files to create one single dataframe
- File parameters


In [39]:
url = "https://pilotformrecognizer.cognitiveservices.azure.com/formrecognizer/documentModels/xfer-GA:analyze?api-version=2022-08-31"

payload=  {'file': open('C:\\Users\hcquinta\\OneDrive - Microsoft\Documents 1\\My MSFT\\[personal]\\chasing\\source\\Catalogo 26-9.pdf', 'rb')}
headers = {
  'Ocp-Apim-Subscription-Key': '44318a526a944baaaba5a6ff726fa20a',
  'Content-Type': 'application/pdf'
}

response = requests.request("POST", url, headers=headers, files=payload)
request_id = response.headers.get('apim-request-id')




{'file': <_io.BufferedReader name='C:\\Users\\hcquinta\\OneDrive - Microsoft\\Documents 1\\My MSFT\\[personal]\\chasing\\document-page1.pdf'>}


In [ ]:
def Inference(FilePName):
    payload=  {'file': open('%s' %filename, 'rb')}
    response = requests.request("POST", url, headers=headers, files=payload)


In [86]:
print(request_id)

cc0b1c9e-64fe-4679-97af-69c6ef2e6fc5


# Get the API Results

Using the returned request id ask the service for the results

TO DO: Loop until result is available

In [87]:


#872b0e87-0b5e-4fbf-b3f2-6bb66f11a6ca
#status = running
# {"status":"running","createdDateTime":"2022-11-10T05:57:29Z","lastUpdatedDateTime":"2022-11-10T05:57:29Z"}
# {'Content-Length': '106', 'Content-Type': 'application/json; charset=utf-8', 'x-envoy-upstream-service-time': '14', 'apim-request-id': '0c88d07b-96b8-4864-936e-897771a2b582', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'South Central US', 'Date': 'Thu, 10 Nov 2022 05:57:30 GMT'}

if response.status_code == 200 or response.status_code == 202:

  url = "https://pilotformrecognizer.cognitiveservices.azure.com/formrecognizer/documentModels/xfer-GA/analyzeResults/" + request_id + "?api-version=2022-08-31"

  payload = "{}"
  headers = {
    'Ocp-Apim-Subscription-Key': '44318a526a944baaaba5a6ff726fa20a',
    'Content-Type': 'application/pdf'
  }  
  
  
  response = requests.request("GET", url, headers=headers, data=payload)
  

print(response.text)
# print(response.headers)



{"status":"succeeded","createdDateTime":"2022-12-06T22:57:50Z","lastUpdatedDateTime":"2022-12-06T22:57:57Z","analyzeResult":{"apiVersion":"2022-08-31","modelId":"xfer-GA","stringIndexType":"textElements","content":"Spyglass Brewing Company\nSpyglass Brewing Company\nTurbulent Flow\n. Denk Fresh\nDouble IPA 8.3% Ak/Vol + 1 Pint\nKeep Coll\nSpyglass Brewing\nTurbulent Flow New England Style Double IPA\nw/ Citra & Rakau hops\nABV 8.3%\n4.4\n$260","pages":[{"pageNumber":1,"angle":0,"width":4.5,"height":3,"unit":"inch","words":[{"content":"Spyglass","polygon":[0.2536,0.9855,0.7568,0.9772,0.759,1.0956,0.2513,1.1011],"confidence":0.995,"span":{"offset":0,"length":8}},{"content":"Brewing","polygon":[0.2592,1.0955,0.4918,1.0935,0.494,1.1588,0.2599,1.1568],"confidence":0.996,"span":{"offset":9,"length":7}},{"content":"Company","polygon":[0.5057,1.0934,0.7608,1.0918,0.7646,1.1595,0.508,1.1588],"confidence":0.996,"span":{"offset":17,"length":7}},{"content":"Spyglass","polygon":[1.3595,1.2227,1.659

In [79]:

print(response.text)


{"status":"succeeded","createdDateTime":"2022-11-11T20:10:36Z","lastUpdatedDateTime":"2022-11-11T20:10:43Z","analyzeResult":{"apiVersion":"2022-08-31","modelId":"xfer-GA","stringIndexType":"textElements","content":"Spyglass Brewing Company\nSpyglass Brewing Company\nTurbulent Flow\n. Denk Fresh\nDouble IPA 8.3% Ak/Vol + 1 Pint\nKeep Coll\nSpyglass Brewing\nTurbulent Flow New England Style Double IPA\nw/ Citra & Rakau hops\nABV 8.3%\n4.4\n$260","pages":[{"pageNumber":1,"angle":0,"width":4.5,"height":3,"unit":"inch","words":[{"content":"Spyglass","polygon":[0.2536,0.9855,0.7568,0.9772,0.759,1.0956,0.2513,1.1011],"confidence":0.995,"span":{"offset":0,"length":8}},{"content":"Brewing","polygon":[0.2592,1.0955,0.4918,1.0935,0.494,1.1588,0.2599,1.1568],"confidence":0.996,"span":{"offset":9,"length":7}},{"content":"Company","polygon":[0.5057,1.0934,0.7608,1.0918,0.7646,1.1595,0.508,1.1588],"confidence":0.996,"span":{"offset":17,"length":7}},{"content":"Spyglass","polygon":[1.3595,1.2227,1.659

## Clean up the response

In [88]:
json_data = json.loads(response.text)['analyzeResult']['documents']

for item in json_data:
    df = pd.DataFrame(item['fields'])

# some cleanup to remove linebreaks
df['style'] = df['style'].replace(to_replace= r'\n', value= ' ', regex=True)

df.loc[['content'],  ['brewery', 'beer', 'style', 'abv', 'untappd', 'price']]
    


,brewery,beer,style,abv,untappd,price
content,Spyglass Brewing,Turbulent Flow New England Style,Double IPA w/ Citra & Rakau hops,8.3%,4.4,$260


## TO DO: Export to Excel File

Export dataframe to excel file

In [22]:
 df.loc[['content'],  ['brewery', 'beer', 'style', 'abv', 'untappd', 'price']].to_csv(r'c:\diad\deleteme.csv', index=False, header=True)
# df.loc[['content'],  ['brewery', 'beer', 'style', 'abv', 'untappd', 'price']].to_excel(r'c:\diad\deleteme.xlsx', index=False, header=True)




,brewery,beer,style,abv,untappd,price
content,Spyglass Brewing,Turbulent Flow New England Style,Double IPA w/ Citra & Rakau hops,8.3%,4.4,$260
